In [1]:
import pandas as pd
import numpy as np
from fidap import fidap_client
fidap = fidap_client(db='bq', api_key="b8a8ebb5f4cf4983a4fa63be6ca418c2")

In [2]:
companies_query = f"""
SELECT MAX(ticker_value) AS ticker, idLabel_value AS article FROM `fidap-301014.fidapschema.wikidata_companies` GROUP BY idLabel_value 
"""

In [3]:
companies_data = fidap.sql(companies_query)

In [4]:
views_query = f"""
SELECT article, date, views FROM `fidap-301014.fidapschema.wikidata_pages` where date > '2020-01-01'
"""

In [5]:
views_data = fidap.sql(views_query)

In [6]:
companies_data = companies_data.merge(views_data, on=['article'])

# Top 10 Stocks By Views

## Over Last Day

In [7]:
companies_data.dropna(inplace=True)

In [8]:
companies_data.sort_values(ascending=False, by=['date', 'views']).head(10)

,ticker,article,date,views
636902,FFIV,F5 Networks,2021-03-12,57205
705915,GOOGL,Google,2021-03-12,27846
151582,AZN,AstraZeneca,2021-03-12,24121
515755,NFLX,Netflix,2021-03-12,21451
557872,AAPL,Apple Inc.,2021-03-12,13503
1048574,NVAX,Novavax,2021-03-12,11568
81678,PFE,Pfizer,2021-03-12,11341
517712,TSLA,"Tesla, Inc.",2021-03-12,10997
1053213,MRNA,Moderna,2021-03-12,9640
136785,JNJ,Johnson & Johnson,2021-03-12,9178


## Over Last Month

Get the current date and then subtract 1 month and then get records which fall in that time frame

In [9]:
from datetime import datetime
from dateutil.relativedelta import relativedelta

In [10]:
now = datetime.date(datetime.now())
start_date = now.strftime("%Y-%m-%d")
end_date = (now - relativedelta(months=1)).strftime("%Y-%m-%d")

Create a Dataframe of only the past months data.

In [11]:
monthly_views = companies_data[(companies_data['date'] < start_date) & (companies_data['date'] >= end_date)]

In [12]:
monthly_views.sort_values(ascending=False, by=['views']).head(10)

,ticker,article,date,views
636885,FFIV,F5 Networks,2021-02-23,73114
636893,FFIV,F5 Networks,2021-03-03,67789
636886,FFIV,F5 Networks,2021-02-24,66845
636888,FFIV,F5 Networks,2021-02-26,65968
636879,FFIV,F5 Networks,2021-02-17,64918
636887,FFIV,F5 Networks,2021-02-25,64519
636884,FFIV,F5 Networks,2021-02-22,64511
636880,FFIV,F5 Networks,2021-02-18,64162
636892,FFIV,F5 Networks,2021-03-02,63915
636899,FFIV,F5 Networks,2021-03-09,63012


## Over Last Year

We would do something like the above, we would get the current date and subtract 1 year and then get all the records in that date range.

In [13]:
now = datetime.date(datetime.now())
start_date = now.strftime("%Y-%m-%d")
end_date = (now - relativedelta(years=1)).strftime("%Y-%m-%d")

In [14]:
yearly_views = companies_data[(companies_data['date'] < start_date) & (companies_data['date'] >= end_date)]

In [15]:
yearly_views.sort_values(ascending=False, by=['views']).head(10)

,ticker,article,date,views
81555,PFE,Pfizer,2020-11-09,302622
81556,PFE,Pfizer,2020-11-10,195289
34793,GME,GameStop,2021-01-28,175970
34794,GME,GameStop,2021-01-29,158155
21207,KNM,Konami,2020-08-25,156739
705690,GOOGL,Google,2020-07-30,132930
217308,NET,Cloudflare,2020-12-12,131548
217307,NET,Cloudflare,2020-12-11,130435
1053097,MRNA,Moderna,2020-11-16,124888
34792,GME,GameStop,2021-01-27,110598


# Top 10 Views/Marketcap - Most Overviewed Stocks

In [16]:
market_cap_query = f"""
SELECT marketcap, ticker FROM `fidap-301014.fidapschema.tickers` where marketcap is not null
"""

In [17]:
market_caps = fidap.sql(market_cap_query)

In [18]:
market_cap_wiki = companies_data.merge(market_caps, on=['ticker'])

In [19]:
market_cap_wiki.dropna(inplace=True)

In [20]:
market_cap_wiki['views_marketcap'] = market_cap_wiki['views']/market_cap_wiki['marketcap']

In [21]:
market_cap_wiki.sort_values(ascending=False, by=['views','views_marketcap']).head(10)

,ticker,article,date,views,marketcap,views_marketcap
47276,PFE,Pfizer,2020-11-09,302622,1.948824e+17,1.552844e-12
47277,PFE,Pfizer,2020-11-10,195289,1.948824e+17,1.002086e-12
19538,GME,GameStop,2021-01-28,175970,1.844810e+16,9.538652e-12
19539,GME,GameStop,2021-01-29,158155,1.844810e+16,8.572970e-12
357089,GOOGL,Google,2020-07-30,132930,1.381980e+18,9.618806e-14
142793,NET,Cloudflare,2020-12-12,131548,2.356340e+16,5.582726e-12
142792,NET,Cloudflare,2020-12-11,130435,2.356340e+16,5.535491e-12
643712,MRNA,Moderna,2020-11-16,124888,5.476440e+16,2.280460e-12
19537,GME,GameStop,2021-01-27,110598,1.844810e+16,5.995089e-12
142791,NET,Cloudflare,2020-12-10,104165,2.356340e+16,4.420627e-12


# Top 10 Increase in Views

## Last Day vs Last Month Average

In [22]:
now = datetime.date(datetime.now())
start_date = now.strftime("%Y-%m-%d")
end_date = (now - relativedelta(months=2)).replace(day=1).strftime("%Y-%m-%d")

In [23]:
monthly_views = companies_data[(companies_data['date'] < start_date) & (companies_data['date'] >= end_date)].copy()

### Get the difference between the current day and previous day based on whether the previous days ticker value macthes the current days.

In [24]:
monthly_views.sort_values(ascending=False, by=['ticker','date'], inplace=True)
monthly_views['prev_day_views'] = np.where(monthly_views['ticker'] == monthly_views['ticker'].shift(-1), monthly_views['views'].shift(-1), np.nan)

### Get the monthly average for all the tickers.

In [25]:
monthly_views['month'] = monthly_views.date.dt.to_period('M')
monthly_av = monthly_views.groupby(['month','ticker'])['views'].mean().to_frame().reset_index()
monthly_av.rename(columns={'views': 'monthly_average'}, inplace=True)

In [26]:
### Merge Tables and Get Top 10 Last Day vs. Monthly Average

In [27]:
monthly_views = monthly_views.merge(monthly_av, on=['month','ticker'])

In [28]:
monthly_views['prev_month_avg'] = monthly_views['prev_day_views']/monthly_views['monthly_average']

In [29]:
monthly_views.sort_values(ascending=False, by=['prev_month_avg']).head(10)

,ticker,article,date,views,prev_day_views,month,monthly_average,prev_month_avg
153086,HRG,HRG Group,2021-02-09,599,14476.0,2021-02,677.607143,21.363411
205016,CPG,Crescent Point Energy,2021-01-12,42,14793.0,2021-01,762.709677,19.395322
90883,OSK,Osisko Mining,2021-02-25,19,15133.0,2021-02,830.250000,18.227040
237868,B,Barnes Group,2021-02-10,34,1284.0,2021-02,77.214286,16.629047
68260,RJF,Raymond James Financial,2021-02-09,1845,27932.0,2021-02,1776.500000,15.723051
182143,ET,Energy Transfer Equity,2021-01-24,116,3459.0,2021-01,233.290323,14.827019
147518,IFL,IOOF,2021-01-17,5,132.0,2021-01,9.166667,14.400000
60182,SBNY,Signature Bank,2021-01-13,2002,6549.0,2021-01,486.096774,13.472626
32411,TLYS,Tillys,2021-01-07,163,1900.0,2021-01,145.677419,13.042516
97921,NVAX,Novavax,2021-01-30,10707,58786.0,2021-01,4585.290323,12.820562
